# Output from example.json

This notebook reads the output generated by running

```
python main.py example.json
```

then converts the HDF5 output file to an xarray (netcdf) object.

In [1]:
import h5py as h5
import xarray as xr
import hvplot.xarray

In [2]:
fns = ['example_run']
rd = {} #results dictionary. I use nested dictionaries to load results from several runs in an organized fashion, e.g. Ideally you do this with the naming of results directory above and loop thorugh what you are changing

for rt in fns:
    print(rt)
    rd[rt] = {}
    ff = f'/Users/vonw/work/software/CommunityFirnModel/CFM_main/CFMoutput_example/csv/CFMresults.hdf5'
    with h5.File(ff,'r') as fn:# Open the CFM results file
        vars_list = list(fn.keys()) # Make a list of the variables stored in the file
        # The first row in each of the model outputs is the initial condition, so we don't want that (i.e. start by using the data in 2nd row)
        # The first column is the time step. So, we use index [1:,1:] on most of the variables to get just the data we want.
        rd[rt]['model_time'] = fn['density'][1:,0] # the first column of the density data is the model time step.
        #rd[rt]['depth'] = fn['depth'][1:,1:] # Put the depth data into a numpy array
        # Change by vpw, 16 January 2024
        rd[rt]['depth'] = fn['depth'][1:] # Put the depth data into a numpy array
        rd[rt]['density'] = fn['density'][1:,1:] # Put the density data into an array
        rd[rt]['temperature'] = fn['temperature'][1:,1:]
        rd[rt]['sig2_d18O'] = fn['iso_sig2_d18O'][1:,1:]
        rd[rt]['sig2_dD'] = fn['iso_sig2_dD'][1:,1:]
        rd[rt]['isotopes_d18O'] = fn['isotopes_d18O'][1:,1:]
        rd[rt]['isotopes_dD'] = fn['isotopes_dD'][1:,1:]
        # rd[rt]['DIP'] = fn['DIP'][1:,1:]


example_run


In [5]:
example_run = xr.Dataset({
                        'density':       (['model_time', 'depth'], rd['example_run']['density']),
                        'temperature':   (['model_time', 'depth'], rd['example_run']['temperature']),
                        'sig2_d18O':     (['model_time', 'depth'], rd['example_run']['sig2_d18O']),
                        'sig2_dD':       (['model_time', 'depth'], rd['example_run']['sig2_dD']),
                        'isotopes_d18O': (['model_time', 'depth'], rd['example_run']['isotopes_d18O']),
                        'isotopes_dD':   (['model_time', 'depth'], rd['example_run']['isotopes_dD']),
                         }, 
                        coords={'model_time': rd['example_run']['model_time'], 'depth': rd['example_run']['depth']})
example_run

<xarray.Dataset>
Dimensions:        (model_time: 492, depth: 481)
Coordinates:
  * model_time     (model_time) float32 1.98e+03 1.98e+03 ... 2.021e+03
  * depth          (depth) float64 0.0 0.25 0.5 0.75 ... 119.2 119.5 119.8 120.0
Data variables:
    density        (model_time, depth) float32 350.0 353.5 369.1 ... nan nan nan
    temperature    (model_time, depth) float32 234.6 235.4 236.1 ... nan nan nan
    sig2_d18O      (model_time, depth) float32 0.0 0.0001434 ... nan nan
    sig2_dD        (model_time, depth) float32 0.0 0.0001227 ... nan nan
    isotopes_d18O  (model_time, depth) float32 -36.7 -26.77 -23.23 ... nan nan
    isotopes_dD    (model_time, depth) float32 -283.6 -203.9 -175.5 ... nan nan

In [6]:
example_run.isotopes_d18O.hvplot()

:Image   [depth,model_time]   (isotopes_d18O)